In [1]:
# append code path
import sys
sys.path.append('../../../code')

# common modules
import numpy as np
from time import time
from swimnetworks import Linear, Dense
from activations import Activations 

# swim solving Hamiltonian systems
from swim import solve_swim_hamiltonian, weight_sampling
from NN.extreme_learning_machine import ELM

# physical systems
from double_nonlinear_pendulum import DoubleNonlinearPendulum

# utils for visualization
from draw import draw_2d


# TODO remove later
from itertools import combinations

## **4-D Example**

The following function is a Hamiltonian with 2 degrees of freedom. Therefore, let's define our general Hamiltonian system!

A Hamiltonian system is determined through a function called ***Hamiltonian***. 

### $$H: E \rightarrow \mathbb{R} $$  

The Hamiltonian incorporates the following set of equations that dictates how a dynamical system changes and evolves over time.  

### $$ \frac{\partial q(t)}{\partial t} = \frac{\partial H(q(t),p(t))}{\partial p} $$

### $$ \frac{\partial p(t)}{\partial t} = -\frac{\partial H(q(t),p(t))}{\partial q} $$

Where $(q,p) \in \mathbb{R}^{2n}$, and $n$ is the number of degrees of freedom of the system. The equations above can be restated as the following to construct a PDE for $H$ at every $(q,p) \in E $,

### $$\begin{bmatrix} 0 & I \\ -I & 0 \end{bmatrix} \cdot \nabla H(q,p) - v(q,p) $$
### $$ = \begin{bmatrix} 0 & I \\ -I & 0 \end{bmatrix} \cdot \begin{bmatrix} \frac{\partial H(q,p)}{\partial q} \\ \frac{\partial H(q,p)}{\partial p} \end{bmatrix} - \begin{bmatrix} \frac{\partial q}{\partial t} \\ \frac{\partial p}{\partial t} \end{bmatrix} $$
### $$ = \begin{bmatrix} \frac{\partial H(q,p)}{\partial p} \\ -\frac{\partial H(q,p)}{\partial q} \end{bmatrix} - \begin{bmatrix} \frac{\partial q}{\partial t} \\ \frac{\partial p}{\partial t} \end{bmatrix} = 0 $$

where $I \in \mathbb{R}^{n \times n}$ is the identity matrix and $v$ is the vector field on $E$. 

### **Nonlinear Double Pendulum**

The particular function that we want to approximate is the nonlinear double pendulum. This system has two degrees of freedom, i.e. $n = 2$, so $(q,p) \in \mathbb{R}^4$ where $q = (q_1, q_2)$ and $p = (p_1, p_2)$, and the PDE for H is as the following:

The Hamiltonian for this particular system is the following:

### $$H: \mathbb{R}^4 \rightarrow \mathbb{R} $$  
### $$ H(q,p) = \frac{p_1^2 + 2 p_{2}^{2} - 2 p_1 p_2 \cos(q_1 - q_2)}{2 (1 + \sin^2(q_1 - q_2))} - 2 \cos(q1) - \cos(q2) $$

### $$ \frac{\partial q_1(t)}{\partial t} = \frac{\partial H(q(t),p(t))}{\partial p_1} = \frac{p_1 - p_2 \cos(q_1 - q_2)}{1 + \sin^{2}(q_1 - q_2)} $$
### $$ \frac{\partial q_2(t)}{\partial t} = \frac{\partial H(q(t),p(t))}{\partial p_2} = \frac{2 p_2 - p_1 \cos(q_1 - q_2)}{1 + \sin^{2}(q_1 - q_2)} $$
### $$ \frac{\partial p_1(t)}{\partial t} = -\frac{\partial H(q(t),p(t))}{\partial q_1} = -2 \sin(q_1) - h_1 + h_2 \sin(2 (q_1 - q_2)) $$
### $$ \frac{\partial p_2(t)}{\partial t} = -\frac{\partial H(q(t),p(t))}{\partial q_2} = -\sin(q_2) + h_1 - h_2 \sin(2 (q_1 - q_2)) $$

In [2]:
double_nonlinear_pendulum = DoubleNonlinearPendulum()
H = double_nonlinear_pendulum.H
dH = double_nonlinear_pendulum.dH

# We will plot the phase space of our example system 
# to get an overview of the global view of the phase 
# space that we would like to approximate.

# num. of points in the grid, grid dimension is (N_q1 x N_q2 x N_p1 x N_p2)
N_q1, N_q2, N_p1, N_p2 = 50, 50, 50, 50
N = N_q1 * N_q2 * N_p1 * N_p2 
q1_lim, q2_lim = [-np.pi,np.pi], [-np.pi, np.pi]
p1_lim, p2_lim = [-np.pi, np.pi], [-np.pi, np.pi]
 
q1_range, q2_range = np.linspace(q1_lim[0], q1_lim[1], N_q1), np.linspace(q2_lim[0], q2_lim[1], N_q2)
p1_range, p2_range = np.linspace(p1_lim[0], p1_lim[1], N_p1), np.linspace(p2_lim[0], p2_lim[1], N_p2)

q1_grid, q2_grid, p1_grid, p2_grid = np.meshgrid(q1_range, q2_range, p1_range, p2_range)

# prepare the input x = [ x_1, .., x_{N} ] of shape (N,4) 
x = np.column_stack([q1_grid.flatten(), q2_grid.flatten(), p1_grid.flatten(), p2_grid.flatten()])
y = H(x) # of shape (N,)
dx = dH(x) # of shape (N, 4)
assert x.shape == (N, 4)
assert y.shape == (N,)
assert dx.shape == (N, 4)

# in order to plot the values, we need to have output of H(x) in
# shape (N_q2, N_q1, N_p1, N_p2), which means we get the value H(x)[i,j,k,l] for the 
# grid point at (q2_grid[i,j,k,l], q1_grid[i,j,k,l], p1_grid[i,j,k,l], p2_grid[i,j,k,l])
print(y.shape)
y_grid = y.reshape((N_q2, N_q1, N_p1, N_p2))
#dx_grid = dx.reshape((N_q2, N_q1, N_p1, N_p2, 4))

# will not be used
del y, dx

(6250000,)


In [3]:
# visualize H(x)
# fix q1 = 0, p1 = 0 by choosing j = N_q1 / 2 and k = N_p1 / 2
#print("y_grid[:,..,..,:] shape:", y_grid[:, N_q1 // 2, N_p1 // 2, :].shape)
#print("N_q1 / 2 == " + str(N_q1 // 2))
#print("y_grid[0, N_q1//2]")

# for 4d poincare maps
def draw_poincare(grid, xrange, yrange, extent, contourlines, xlabel, ylabel, aspect, title, fixed_dims={"q1": 0, "q2":0}, save=False, verbose=True):
    fixed_dims_names = list(fixed_dims.keys())
    fixed_dims_values = list(fixed_dims.values())
    title = title + "\n" + "with " + fixed_dims_names[0] + "=" + str(fixed_dims_values[0]) + "," + fixed_dims_names[1] + "=" + str(fixed_dims_values[1])
    if save:
        save = "./plots/" + title.replace("\n", "_").replace("=", "_").replace(",", "_").replace(" ", "_") + ".pdf"  
    else:
        save = ""
    draw_2d(grid, xrange, yrange, extent=extent, contourlines=contourlines, xlabel=xlabel, ylabel=ylabel, aspect=aspect, title=title, save=save, verbose=verbose)

def draw_poincares(grid, title, save=False, verbose=True):
    draw_poincare(grid[:, N_q1//2, N_p1//2, :].reshape((N_p2, N_q2)), q2_range, p2_range, [q2_lim[0],q2_lim[1],p2_lim[0],p2_lim[1]], 50, "q2", "p2", 1, 
                  title, { "q1": q1_range[N_q1//2], "p1": p1_range[N_p1//2] }, save=True, verbose=verbose)

    draw_poincare(grid[N_q2//2, :, :, N_p2//2].reshape((N_p1, N_q1)), q1_range, p1_range, [q1_lim[0],q1_lim[1],p1_lim[0],p1_lim[1]], 50, "q1", "p1", 1, 
                  title, { "q2": q2_range[N_q2//2], "p2": p2_range[N_p2//2] }, save=True, verbose=verbose)

    draw_poincare(grid[:, :, N_p1//2, N_p2//2].reshape((N_q1, N_q2)), q2_range, q1_range, [q2_lim[0],q2_lim[1],q1_lim[0],q1_lim[1]], 50, "q2", "q1", 1, 
                  title, { "p1": p1_range[N_p1//2], "p2": p2_range[N_p2//2] }, save=True, verbose=verbose)

    draw_poincare(grid[N_q2//2, N_q1//2, :, :].reshape((N_p2, N_p1)), p1_range, p2_range, [p1_lim[0],p1_lim[1],p2_lim[0],p2_lim[1]], 50, "p1", "p2", 1, 
                  title, { "q1": q1_range[N_q1//2], "q2": q2_range[N_q2//2] }, save=True, verbose=verbose)

    draw_poincare(grid[40, 0, :, :].reshape(N_p2, N_p1), p1_range, p2_range, [p1_lim[0],p1_lim[1],p2_lim[0],p2_lim[1]], 50, "p1", "p2", 1,
                  title, { "q2": q2_range[40], "q1": q1_range[0] }, save=True, verbose=verbose)

    draw_poincare(grid[40, :, 8, :].reshape((N_p2, N_q1)), q1_range, p2_range, [q1_lim[0],q1_lim[1],p2_lim[0],p2_lim[1]], 50, "q1", "p2", 1,
                  title, { "q2": q2_range[40], "p1": p1_range[8] }, save=True, verbose=verbose)

    draw_poincare(grid[0, :, 0, :].reshape((N_p2, N_q1)), q1_range, p2_range, [q1_lim[0],q1_lim[1],p2_lim[0],p2_lim[1]], 50, "q1", "p2", 1, 
                  title, { "q2": q2_range[0], "p1": p1_range[0] }, save=True, verbose=verbose)

draw_poincares(y_grid, "Original Double Pendulum Hamiltonian", save=True, verbose=False)

In [4]:
np.random.seed(123495)

# Sample training dataset
# hint: use sampled 1000 random points in [-pi/2,pi/2]x[-1/2,1/2]
#noise = 1e-5
noise = 0 # test with noise later TODO
#K_q1, K_q2, K_p1, K_p2 = 25, 25, 5, 5 # 15625
K_q1, K_q2, K_p1, K_p2 = 10, 10, 10, 10 # 10,000 points
K = K_q1 * K_q2 * K_p1 * K_p2
q1_lim_train, q2_lim_train = [-np.pi, np.pi], [-np.pi, np.pi]
p1_lim_train, p2_lim_train = [-np.pi, np.pi], [-np.pi, np.pi]
q1_range_train, q2_range_train = np.linspace(q1_lim_train[0], q1_lim_train[1], K_q1)+np.random.randn(K_q1)*noise*1e-3, np.linspace(q2_lim_train[0], q2_lim_train[1], K_q2)+np.random.randn(K_q1)*noise*1e-3
p1_range_train, p2_range_train = np.linspace(p1_lim_train[0], p1_lim_train[1], K_p1)+np.random.randn(K_p1)*noise*1e-3, np.linspace(p2_lim_train[0], p2_lim_train[1], K_p2)+np.random.randn(K_p2)*noise*1e-3
q1_grid_train, q2_grid_train, p1_grid_train, p2_grid_train = np.meshgrid(q1_range_train, q2_range_train, p1_range_train, p2_range_train) 
x_train_values = np.column_stack([q1_grid_train.flatten(), q2_grid_train.flatten(), p1_grid_train.flatten(), p2_grid_train.flatten()])
x_train_derivs = dH(x_train_values) # of shape (K,4)
x0 = np.array([0,0,0,0])
f0 = H(x0.reshape(-1,4))

# Sample points that we want to predict (same as above plot) 
q1_lim_pred, q2_lim_pred = [-np.pi,np.pi], [-np.pi, np.pi]
p1_lim_pred, p2_lim_pred = [-np.pi,np.pi], [-np.pi, np.pi]
q1_range_pred, q2_range_pred = q1_range+np.random.randn(N_q1)*noise*1e-3, q2_range+np.random.randn(N_q2)*noise*1e-3 
p1_range_pred, p2_range_pred = p1_range+np.random.randn(N_p1)*noise*1e-3, p2_range+np.random.randn(N_p2)*noise*1e-3 

q1_grid_pred, q2_grid_pred, p1_grid_pred, p2_grid_pred = np.meshgrid(q1_range_pred, q2_range_pred, p1_range_pred, p2_range_pred)
x_pred = np.column_stack([q1_grid_pred.flatten(), q2_grid_pred.flatten(), p1_grid_pred.flatten(), p2_grid_pred.flatten()])

y_true = H(x) # of shape (N,) WITHOUT THE NOISE

In [15]:
def solve(title, random_seed=1, M=128, regularization_scale=1e-8, sample_uniformly=True, full_swim=False, parameter_sampler="relu", activation="relu", verbose=True):
    if full_swim:
        # swim model with approximation, first train with noisy data
        swim_model = weight_sampling(x_train_values, x_train_derivs, x0, f0,
                                     n_hidden=M, activation=activation, parameter_sampler=parameter_sampler, rcond=regularization_scale, random_seed=1 + random_seed + 1 * 1234, 
                                     include_bias=True)
    else:
        swim_model = solve_swim_hamiltonian(x_train_values, x_train_derivs, x0, f0,
                                            n_hidden=M, activation=activation, parameter_sampler=parameter_sampler, sample_uniformly=sample_uniformly, rcond=regularization_scale, random_seed=1 + random_seed + 1 * 1234, 
                                            include_bias=True)

    # perform approximation on clean data
    phi_2 = swim_model.transform(x).reshape(-1) # (N)

    # train loss on the points we have trained (noisy data)
    train_mse_loss = np.sum((H(x_train_values).reshape(-1) - swim_model.transform(x_train_values).reshape(-1))**2) / K
        
    # eval loss on clean data
    eval_mse_loss = np.sum((y_true - phi_2)**2) / N
        
    print(title + " -> train loss:", str(train_mse_loss))
    print(title + " -> eval loss:", str(eval_mse_loss))

    phi_2 = phi_2.reshape((N_q2, N_q1, N_p1, N_p2))
    draw_poincares(phi_2, title, save=True, verbose=verbose)

In [16]:
solve("ELM (relu) Double Pendulum Hamiltonian", parameter_sampler="random", activation="relu", full_swim=False, verbose=False)

ELM (relu) Double Pendulum Hamiltonian -> train loss: 8.21177978215589
ELM (relu) Double Pendulum Hamiltonian -> eval loss: 6.567994861847643


In [17]:
solve("SWIM (relu) with Random Sampling Double Pendulum Hamiltonian", parameter_sampler="relu", activation="relu", full_swim=False, verbose=False)

SWIM (relu) with Random Sampling Double Pendulum Hamiltonian -> train loss: 17.051978671818624
SWIM (relu) with Random Sampling Double Pendulum Hamiltonian -> eval loss: 14.506729733836327


In [18]:
solve("SWIM (relu) with Approximation Double Pendulum Hamiltonian", parameter_sampler="relu", activation="relu", full_swim=True, verbose=False)

SWIM (relu) with Approximation Double Pendulum Hamiltonian -> train loss: 9.989376564211394
SWIM (relu) with Approximation Double Pendulum Hamiltonian -> eval loss: 7.924377431842241


In [19]:
solve("ELM (tanh) Double Pendulum Hamiltonian", parameter_sampler="random", activation="tanh", full_swim=False, verbose=False)

ELM (tanh) Double Pendulum Hamiltonian -> train loss: 15.591729896647827
ELM (tanh) Double Pendulum Hamiltonian -> eval loss: 14.387824426876561


In [20]:
solve("SWIM (tanh) with Random Sampling Double Pendulum Hamiltonian", parameter_sampler="tanh", activation="tanh", full_swim=False, verbose=False)

SWIM (tanh) with Random Sampling Double Pendulum Hamiltonian -> train loss: 6.399127243199689
SWIM (tanh) with Random Sampling Double Pendulum Hamiltonian -> eval loss: 4.74277093933671


In [21]:
solve("SWIM (tanh) with Approximation Double Pendulum Hamiltonian", parameter_sampler="tanh", activation="tanh", full_swim=True, verbose=False)

SWIM (tanh) with Approximation Double Pendulum Hamiltonian -> train loss: 6.814292130660309
SWIM (tanh) with Approximation Double Pendulum Hamiltonian -> eval loss: 5.308832460912402


In [22]:
solve("ELM (sigmoid) Double Pendulum Hamiltonian", parameter_sampler="random", activation="sigmoid", full_swim=False, verbose=False)

ELM (sigmoid) Double Pendulum Hamiltonian -> train loss: 17.160630293818386
ELM (sigmoid) Double Pendulum Hamiltonian -> eval loss: 16.0005504719683


In [23]:
solve("SWIM (sigmoid) with Random Sampling Double Pendulum Hamiltonian", parameter_sampler="tanh", activation="sigmoid", full_swim=False, verbose=False)

SWIM (sigmoid) with Random Sampling Double Pendulum Hamiltonian -> train loss: 6.842137589703514
SWIM (sigmoid) with Random Sampling Double Pendulum Hamiltonian -> eval loss: 4.960105374043485


In [24]:
solve("SWIM (sigmoid) with Approximation Double Pendulum Hamiltonian", parameter_sampler="tanh", activation="sigmoid", full_swim=True, verbose=False)

SWIM (sigmoid) with Approximation Double Pendulum Hamiltonian -> train loss: 6.7162859420612175
SWIM (sigmoid) with Approximation Double Pendulum Hamiltonian -> eval loss: 5.058384934448995
